<h1>Scraping Fidelity.com</h1>
Retrieve the latest sector performance data from the US markets, and get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by,<strong> decreasing order of market weight <strong>. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    if not response.status_code == 200:
        return None
    
    sector_list = list()
    results_page = BeautifulSoup(response.content,'lxml')
    sector_names = results_page.find_all('a', class_="heading1")
    for sector in sector_names:
        sector_name = sector.get_text()
        sector_link = "https://eresearch.fidelity.com" + sector['href']
        sector_list.append((sector_name, sector_link))
    for sector in (sector_list):
        url = sector[1]
        sector_change,sector_market_cap,sector_market_weight = get_sector_change_and_market_cap(url)
        output_list.append((sector[0].strip(),sector_change,sector_market_cap,sector_market_weight, sector[1])) #Sector_list is a list of tuples of (sector name, sector_url)
    output_list.sort(key=lambda k: k[3], reverse = True)
    return output_list
    

In [2]:
def get_sector_change_and_market_cap(sector_page_link):
    
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(sector_page_link)
    if not response.status_code == 200:
        return None
    results_page = BeautifulSoup(response.content, 'lxml')
    for_regex = results_page.find('table', class_="snapshot-data-tbl").get_text()
    market_capRegex = re.compile(r'\$\d+(?:\.\d+\w)?')
    sector_market_cap = market_capRegex.findall(for_regex)[0]
    change_and_weight_Regex = re.compile(r'\-?\d+(?:\.\d+\w)%')
    sector_change = float(change_and_weight_Regex.findall(for_regex)[0][:-1])
    sector_market_weight = float(change_and_weight_Regex.findall(for_regex)[1][:-1])
    return sector_change,sector_market_cap,sector_market_weight

In [3]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

(1.33, '$6.80T', 9.93)

In [4]:
get_us_sector_performance()

[('Information Technology',
  0.22,
  '$12.21T',
  26.1,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.67,
  '$7.49T',
  14.44,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Financials',
  0.46,
  '$8.57T',
  12.88,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Consumer Discretionary',
  1.33,
  '$6.80T',
  9.93,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Industrials',
  0.58,
  '$5.55T',
  8.59,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Communication Services',
  1.06,
  '$4.50T',
  8.27,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Co

In [56]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)
"""
[('Telecommunication Services',
  0.21,
  '$1.74T',
  2.0,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Materials',
  0.22,
  '$1.95T',
  2.49,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
 ('Real Estate',
  -0.45,
  '$1.22T',
  2.7,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  -0.33,
  '$1.25T',
  2.86,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Energy',
  0.76,
  '$3.90T',
  5.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Consumer Staples',
  -0.32,
  '$3.58T',
  6.8,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Industrials',
  0.72,
  '$4.31T',
  9.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Discretionary',
  1.03,
  '$5.76T',
  12.9,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.39,
  '$7.45T',
  13.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Health Care',
  0.76,
  '$5.70T',
  14.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Information Technology',
  0.81,
  '$9.84T',
  26.17,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
"""

"\n[('Telecommunication Services',\n  0.21,\n  '$1.74T',\n  2.0,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),\n ('Materials',\n  0.22,\n  '$1.95T',\n  2.49,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),\n ('Real Estate',\n  -0.45,\n  '$1.22T',\n  2.7,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),\n ('Utilities',\n  -0.33,\n  '$1.25T',\n  2.86,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),\n ('Energy',\n  0.76,\n  '$3.90T',\n  5.83,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),\n ('Consumer Staples',\n  -0.32,\n  '$3.58T',\n  6.8,\n  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'

<h3>Sorting</h3>

In [127]:
x = [('a',23.2,'b'),('c',17.4,'f'),('e',1.74,'bb'),('d',29.2,'z')]
x.sort() #Sorts by the first element of the tuple
x

[('a', 23.2, 'b'), ('c', 17.4, 'f'), ('d', 29.2, 'z'), ('e', 1.74, 'bb')]

In [128]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x

[('e', 1.74, 'bb'), ('c', 17.4, 'f'), ('a', 23.2, 'b'), ('d', 29.2, 'z')]